In [1]:
import numpy as np
from ondelette import wavelets
import pywt
import tensorflow as tf

rng = np.random.default_rng()

2025-02-19 20:38:22.352974: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
sample_size = 1000

samples = rng.integers(low=-32768, high=32767, size=sample_size, dtype=np.int16, endpoint=True)
tf_samples = tf.constant(samples, shape=[1, sample_size, 1], dtype=tf.float64)  # Only NWC on CPU

for name, wavelet in wavelets.all.items():
    wavelet = wavelets.all[name](tf.float64)
    filters = wavelets.dwt_filters(wavelet)
    wavelet_width = wavelet.shape[0]

    dec_conv = wavelets.dwt(filters, tf_samples)
    dec = tf.reshape(dec_conv, [2, -1])

    pywt_wavelet = pywt.Wavelet(name)
    pywt_dec_lo, pywt_dec_hi = pywt.dwt(samples, pywt_wavelet, 'zero')
    pywt_dec_all = np.array([pywt_dec_lo, pywt_dec_hi])
    odd = samples.shape[0] % 2
    chop = (wavelet_width - 2) // 2
    if chop:
        pywt_dec = pywt_dec_all[..., chop:-(chop + odd)]
    elif odd:
        pywt_dec = pywt_dec_all[..., chop:-odd]
    else:
        pywt_dec = pywt_dec_all

    # print(f'{name}:')
    # print(f'our shape: {dec.shape}')
    # print(f'their shape: {pywt_dec.shape}')
    diff = np.max(np.abs(dec.numpy() - pywt_dec))
    print(f'{name} max diff: {diff}')


bior1.1 max diff: 0.0
bior1.3 max diff: 1.4551915228366852e-11
bior1.5 max diff: 1.4551915228366852e-11
bior2.2 max diff: 1.4551915228366852e-11
bior2.4 max diff: 1.4551915228366852e-11
bior2.6 max diff: 1.4551915228366852e-11
bior2.8 max diff: 1.4551915228366852e-11
bior3.1 max diff: 1.4551915228366852e-11
bior3.3 max diff: 1.4551915228366852e-11
bior3.5 max diff: 2.1827872842550278e-11
bior3.7 max diff: 2.1827872842550278e-11
bior3.9 max diff: 2.9103830456733704e-11
bior4.4 max diff: 1.4551915228366852e-11
bior5.5 max diff: 2.1827872842550278e-11
bior6.8 max diff: 1.4551915228366852e-11
coif1 max diff: 1.4551915228366852e-11
coif2 max diff: 1.4551915228366852e-11
coif3 max diff: 2.1827872842550278e-11
coif4 max diff: 2.9103830456733704e-11
coif5 max diff: 2.9103830456733704e-11
coif6 max diff: 3.637978807091713e-11
coif7 max diff: 2.9103830456733704e-11
coif8 max diff: 3.637978807091713e-11
coif9 max diff: 3.637978807091713e-11
coif10 max diff: 4.3655745685100555e-11
coif11 max diff: